# Demonstration of using Curve registry

### Essential links:
* Source code: <https://github.com/curvefi/curve-pool-registry/tree/b17>;
* ABI: <https://github.com/curvefi/curve-pool-registry/blob/b17/deployed/2020-06-20/registry.abi>;
* Registry contract: `0x7002B727Ef8F5571Cb5F9D70D13DBEEb4dFAe9d1`.

### Complimentary information for tests:
* Y pool: `0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51` (underlying coins: DAI/USDC/USDT/TUSD, compounding coins: yDAI/yUSDC/yUSDT/yTUSD);
* DAI: `0x6B175474E89094C44Da98b954EedeAC495271d0F`;
* USDT: `0xdAC17F958D2ee523a2206206994597C13D831ec7`;

In [1]:
# Init Brownie environment
# <https://eth-brownie.readthedocs.io/en/stable/python-package.html#accessing-the-network>
from brownie import network, Contract
network.connect('mainnet')

In [2]:
import json
with open('registry.abi', 'r') as f:
    abi = json.load(f)
registry = Contract.from_abi('CurveRegistry', '0x7002B727Ef8F5571Cb5F9D70D13DBEEb4dFAe9d1', abi)

### Finding a pool by coins
Pools can be found for a given pair (from -> to). There is an array of pools per pair on chain, and you need to pass an integer (0, 1, 2...) to iterate over all pools in existence who have this currency pair.
Once there are no more pools existing for that pair, zero address is returned.

In [3]:
registry.find_pool_for_coins("0xdAC17F958D2ee523a2206206994597C13D831ec7",
                             "0x6B175474E89094C44Da98b954EedeAC495271d0F", 0)

'0x52EA46506B9CC5Ef470C5bf89f17Dc28bB35D85C'

In [4]:
registry.find_pool_for_coins("0xdAC17F958D2ee523a2206206994597C13D831ec7",
                             "0x6B175474E89094C44Da98b954EedeAC495271d0F", 1)

'0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51'

In [5]:
registry.find_pool_for_coins("0xdAC17F958D2ee523a2206206994597C13D831ec7",
                             "0x6B175474E89094C44Da98b954EedeAC495271d0F", 10)  # ... eventually we hit 0x0000

'0x0000000000000000000000000000000000000000'

### Getting pool's coins
Information is given in format:
```python
[
    (compounding_coin, compounding_coin, ...),
    (underlying_coin, underlying_coin, ...),
    (compounding_coin_decimals, compounding_coin_decimals, ...),
    (underlying_coin_decimals, underlying_coin_decimals, ...)
]
```

In [6]:
registry.get_pool_coins('0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51')

(('0x16de59092dAE5CcF4A1E6439D611fd0653f0Bd01',
  '0xd6aD7a6750A7593E092a9B218d66C0A814a3436e',
  '0x83f798e925BcD4017Eb265844FDDAbb448f1707D',
  '0x73a052500105205d34Daf004eAb301916DA8190f',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000'),
 ('0x6B175474E89094C44Da98b954EedeAC495271d0F',
  '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
  '0xdAC17F958D2ee523a2206206994597C13D831ec7',
  '0x0000000000085d4780B73119b644AE5ecd22b376',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000'),
 (18, 6, 6, 18, 0, 0, 0, 0),
 (18, 6, 6, 18, 0, 0, 0, 0))

### Pool info
Format:
```python
[
    (balance_0, balance_1, ...),
    (underlying_0, underlying_1, ...),
    (*precisions_for_compounding_coins ...),
    (*precisions_for_underlying_coins ...),
    pool_token_address,
    amplification,
    fee multiplied by 1e10
]
```

In [7]:
registry.get_pool_info('0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51')

((745449874799640907867105,
  2042421414825,
  8925850569394,
  2860775568344733897581759,
  0,
  0,
  0,
  0),
 (758741938521378355217029,
  2360734356331,
  9145017193056,
  2901118428965827740704506,
  0,
  0,
  0,
  0),
 (18, 6, 6, 18, 0, 0, 0, 0),
 (18, 6, 6, 18, 0, 0, 0, 0),
 '0xdF5e0e81Dff6FAF3A7e52BA697820c5e32D806A8',
 2000,
 4000000)

With these rates, underlying_balance = balance * rate / 1e18

In [8]:
registry.get_pool_rates('0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51')

(1017830928907607686,
 1155850765760560737,
 1024554144387495179,
 1014102071154234767,
 0,
 0,
 0,
 0)

### Estimate the most probable gas spent for exchanging two coins

In [9]:
registry.estimate_gas_used('0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51',
                           '0xdAC17F958D2ee523a2206206994597C13D831ec7',
                           '0x6B175474E89094C44Da98b954EedeAC495271d0F')

799000

### Calculate exchange amount(s)
We're swapping USDC (6 digits) to DAI (18 digits)

In [10]:
dai_amount = registry.get_exchange_amount(
                             '0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51',
                             '0xdAC17F958D2ee523a2206206994597C13D831ec7',
                             '0x6B175474E89094C44Da98b954EedeAC495271d0F',
                             10 ** 6)  # Dump 1 USDC
dai_amount / 1e18  # DAI has 18 digits

0.989768806131727

How much USDC do we need to get 1 DAI?

In [11]:
usdc_amount = registry.get_input_amount(
                             '0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51',
                             '0xdAC17F958D2ee523a2206206994597C13D831ec7',
                             '0x6B175474E89094C44Da98b954EedeAC495271d0F',
                             10 ** 18)
usdc_amount / 1e6

1.010336

Get many exchange amounts (DAI for USDC) at once

In [12]:
amounts = registry.get_exchange_amounts(
                             '0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51',
                             '0xdAC17F958D2ee523a2206206994597C13D831ec7',
                             '0x6B175474E89094C44Da98b954EedeAC495271d0F',
                             [x * 10 ** 6 for x in range(1, 101)])
[x / 1e18 for x in amounts][:5]  # Let's show only first 5 out of 100

[0.9897688061312254,
 1.97953758668347,
 2.969306341656635,
 3.9590750710506226,
 4.9488437748653356]

### Exchanges using the registry

Reconnect to a fork of mainnet on ganache-cli

In [13]:
network.disconnect()
network.connect('mainnet-fork')

Launching 'ganache-cli --port 8545 --gasLimit 10000000 --accounts 10 --hardfork istanbul --mnemonic brownie --fork https://mainnet.infura.io/v3/fac98e56ea7e49608825dfc726fab703'...


Create an account using a saved plaintext private key (never do that at home!)

In [14]:
from brownie import accounts
from test_address import private_key
alice = accounts.add(bytes.fromhex(private_key))
alice

<LocalAccount '0x9d940795985176230F18ED4ca64545C1886615F6'>

Make ERC20 contract objects to check balances before and after

In [15]:
with open('erc20.abi', 'r') as f:
    abi = json.load(f)
dai = Contract.from_abi('ERC20', '0x6B175474E89094C44Da98b954EedeAC495271d0F', abi)
usdc = Contract.from_abi('ERC20', '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', abi)

In [16]:
dai.balanceOf(alice)/1e18

0.0

In [17]:
usdc.balanceOf(alice)/1e6

5.0

In [18]:
usdc.approve(registry, 5 * 10 ** 6, {'from': alice})

Transaction sent: 0x10423dd0e769e3816ee64fae28622820eb9195dc1bef3c60211a125601b1b4ad
  Gas price: 0.0 gwei   Gas limit: 6721975
  ERC20.approve confirmed - Block: 10309099   Gas used: 34593 (0.51%)



<Transaction '0x10423dd0e769e3816ee64fae28622820eb9195dc1bef3c60211a125601b1b4ad'>

In [19]:
registry.exchange('0xA5407eAE9Ba41422680e2e00537571bcC53efBfD',  # SUSD pool
                  '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',  # from USDC
                  '0x6B175474E89094C44Da98b954EedeAC495271d0F',  # to DAI
                  10 ** 6,                                       # swap 1 dollar
                  10 ** 18 // 2,                                 # require no less than half a dollar
                  {'from': alice})

Transaction sent: 0xfef8f2f9d975175b584142f12e84f91bf88a1a25a12fb9f77622bcb30f679d42
  Gas price: 0.0 gwei   Gas limit: 6721975
  CurveRegistry.exchange confirmed - Block: 10309100   Gas used: 133636 (1.99%)



<Transaction '0xfef8f2f9d975175b584142f12e84f91bf88a1a25a12fb9f77622bcb30f679d42'>

In [20]:
dai.balanceOf(alice)/1e18

0.992977248517724

In [21]:
usdc.balanceOf(alice)/1e6

4.0